In [1]:
import pandas as pd
df = pd.read_csv('CLEANED_FMCG_2022_2024.csv')
df.head()

,date,sku,brand,segment,category,channel,region,pack_type,price_unit,promotion_flag,delivery_days,stock_available,delivered_qty,units_sold
0,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-Central,Multipack,2.38,0,1,141,128,9
1,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-North,Single,1.55,1,3,0,129,0
2,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Retail,PL-South,Carton,4.00,0,5,118,161,8
3,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Discount,PL-Central,Single,5.16,0,2,81,114,7
4,2022-01-21,MI-006,MiBrand1,Milk-Seg3,Milk,Discount,PL-North,Single,7.66,0,4,148,204,12


you want us to drill down into Aug 2023 specifically (e.g., top SKUs, regions, channels for that peak month) to uncover why sales exploded? That could directly connect your EDA with Step 2: Inventory Optimization tomorrow.

Deep Dive Plan – August 2023

Filter dataset → keep only rows from Aug 2023.

In [2]:
aug_2023 = df[(df['date'] >= '2023-8-1') & (df['date'] <= '2023-8-31')]

Top SKUs → find which products drove sales.

In [6]:
aug_2023.groupby('sku')['units_sold'].sum().sort_values(ascending=False)


Series([], Name: units_sold, dtype: int64)

Top Brands/Categories → maybe one category (like Milk or Snacks) dominated.

In [7]:
aug_2023.groupby('category')['units_sold'].sum().sort_values(ascending=False).head()

Series([], Name: units_sold, dtype: int64)

Channels → check whether Retail, Discount, or Online channels contributed most.

In [9]:
aug_2023.groupby('channel')['units_sold'].sum()

Series([], Name: units_sold, dtype: int64)

Regions → see which part of the country drove the spike.

In [10]:
aug_2023.groupby('region')['units_sold'].sum(
).sort_values(ascending=False).head()

Series([], Name: units_sold, dtype: int64)

🧠 Business Angle

Once we run those, we can answer:

Was the peak driven by a few SKUs (blockbuster products)?

Did a brand/category do something special (promo, price cut)?

Was it regional (e.g., North booming)?

Or did every channel rise together (a company-wide push)?